In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io

In [3]:
path_to_zip = keras.utils.get_file('spa-eng.zip', origin='https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip', extract=True)

2638744/2638744 [==============================] - 0s 0us/step


In [4]:
path_to_zip

'/root/.keras/datasets/spa-eng.zip'

In [5]:
path_to_file=os.path.join(os.path.dirname(path_to_zip),'spa-eng','spa.txt')
path_to_file

'/root/.keras/datasets/spa-eng/spa.txt'

In [6]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [7]:
def preprocess_senetence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.~,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.~,]+", " ", w)
    w = w.rstrip().strip()
    w = '<strat> ' + w + ' <end>'
    return w

In [8]:
en_sentence = "I have some friends to help."
preprocess_senetence(en_sentence)

'<strat> i have some friends to help . <end>'

In [9]:
sp_sentence = "Aún no he dicho nada."
preprocess_senetence(sp_sentence)

'<strat> aun no he dicho nada . <end>'

In [10]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_senetence(w) for w in l.split('\t')] for l in lines[:num_examples]]
    return zip(*word_pairs)

In [11]:
print(create_dataset(path_to_file, 2))

In [12]:
en, sp = create_dataset(path_to_file, None)

In [13]:
len(en),len(sp)

(118964, 118964)

In [14]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [15]:
def max_length(tensor):
    return max(len(t) for t in tensor)

equal = to words


In [16]:
def tokenize(lang):
    lang_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

In [17]:
def load_dataset(path, num_examples=None):
    targ_lang, inp_lang = create_dataset(path, num_examples)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, target_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, target_lang_tokenizer

In [18]:
input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer = load_dataset(path_to_file, 20000)

In [19]:
input_tensor

array([[  1,  97,   3, ...,   0,   0,   0],
       [  1, 179,   3, ...,   0,   0,   0],
       [  1, 466,   3, ...,   0,   0,   0],
       ...,
       [  1,   7,  10, ...,   0,   0,   0],
       [  1,   7,  10, ...,   0,   0,   0],
       [  1,   7,  10, ...,   0,   0,   0]], dtype=int32)

In [20]:
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [22]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print(t, ' .... ', lang.index_word[t])

In [23]:
convert(input_lang_tokenizer, input_tensor[0])

1  ....  <strat>
97  ....  ve
3  ....  .
2  ....  <end>


In [24]:
input_tensor[0]

array([ 1, 97,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [25]:
target_lang_tokenizer.word_index

{'<strat>': 1,
 '<end>': 2,
 '.': 3,
 'i': 4,
 'you': 5,
 '?': 6,
 'tom': 7,
 'is': 8,
 'it': 9,
 's': 10,
 'a': 11,
 'he': 12,
 't': 13,
 'the': 14,
 'we': 15,
 'm': 16,
 'me': 17,
 're': 18,
 'that': 19,
 'this': 20,
 'to': 21,
 'do': 22,
 'are': 23,
 'can': 24,
 'my': 25,
 'they': 26,
 'was': 27,
 'she': 28,
 'don': 29,
 'have': 30,
 'your': 31,
 'go': 32,
 'what': 33,
 'in': 34,
 'not': 35,
 'll': 36,
 'like': 37,
 'here': 38,
 'on': 39,
 'him': 40,
 'let': 41,
 'be': 42,
 'did': 43,
 'know': 44,
 'come': 45,
 'up': 46,
 'am': 47,
 'want': 48,
 'how': 49,
 ',': 50,
 'mary': 51,
 'get': 52,
 'who': 53,
 'very': 54,
 'now': 55,
 'need': 56,
 'has': 57,
 'please': 58,
 'no': 59,
 'there': 60,
 'help': 61,
 'her': 62,
 'love': 63,
 'at': 64,
 'see': 65,
 'just': 66,
 'out': 67,
 'his': 68,
 've': 69,
 'got': 70,
 'for': 71,
 'where': 72,
 'look': 73,
 'stop': 74,
 'one': 75,
 'us': 76,
 'good': 77,
 'car': 78,
 'too': 79,
 'so': 80,
 'all': 81,
 'why': 82,
 'will': 83,
 'an': 84,
 'hom

In [26]:
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 64
steps_per_epoch = len(X_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index) + 1
vocab_targ_size = len(target_lang_tokenizer.word_index) + 1

convert to tensorflow

In [27]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [28]:
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    def initilize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))


In [29]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [30]:
encoder

In [31]:
simple_hidden = encoder.initilize_hidden_state()
simple_hidden

<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [32]:
example_input_batch, example_target_batch = next(iter(dataset))

In [33]:
encoder(example_input_batch, simple_hidden)

(<tf.Tensor: shape=(64, 16, 1024), dtype=float32, numpy=
 array([[[-1.3867304e-02, -1.4910647e-02, -2.7736884e-03, ...,
           6.5706889e-03, -8.6802800e-05,  2.2787252e-03],
         [-3.8181278e-03,  3.0817572e-04, -3.9349422e-03, ...,
          -5.1668617e-03, -3.2297852e-03, -1.0563156e-02],
         [-1.4705049e-02,  1.1103795e-02, -7.9958886e-03, ...,
          -3.2451171e-03, -1.9584314e-03, -1.1390633e-02],
         ...,
         [ 1.9635055e-02,  5.1810266e-03,  4.0658031e-04, ...,
           8.1495903e-03, -1.5585002e-02, -1.5376742e-02],
         [ 1.9614244e-02,  4.9648858e-03,  6.9931068e-04, ...,
           8.2933428e-03, -1.5761487e-02, -1.5432717e-02],
         [ 1.9578893e-02,  4.8386822e-03,  8.9701317e-04, ...,
           8.3865533e-03, -1.5850341e-02, -1.5463739e-02]],
 
        [[-1.3867304e-02, -1.4910647e-02, -2.7736884e-03, ...,
           6.5706889e-03, -8.6802800e-05,  2.2787252e-03],
         [-9.5584244e-03,  3.7205432e-04, -3.9382512e-03, ...,
         

In [34]:
simple_output, simple_states = encoder(example_input_batch, simple_hidden)

In [35]:
class Attention(keras.layers.Layer):
    def __init__ (self, units):
        super(Attention, self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
    def call (self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        atteion_weights = tf.nn.softmax(score, axis=1)
        context_vector = atteion_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, atteion_weights

In [36]:
attention_layer = Attention(10)
attention_layer(simple_hidden, simple_output)

(<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[ 7.1667298e-03,  5.3439499e-03, -2.2248339e-03, ...,
          2.4999985e-03, -7.2825309e-03, -9.5850490e-03],
        [ 6.8199532e-03,  5.1684859e-03, -1.5664676e-03, ...,
          2.6855646e-03, -6.2899645e-03, -7.0009045e-03],
        [ 1.0203638e-02,  4.2281952e-03, -9.1903516e-05, ...,
          4.8851687e-03, -1.0887261e-02, -8.4632635e-03],
        ...,
        [ 7.1569751e-03,  3.1073373e-03, -8.8585378e-04, ...,
          8.5433722e-03, -7.4149831e-03, -8.0103474e-03],
        [ 8.6027756e-03,  2.4226005e-03, -2.1138897e-03, ...,
          4.5032171e-03, -8.9810733e-03, -8.8280383e-03],
        [ 9.5744897e-03,  1.8695681e-03, -1.1786344e-03, ...,
          1.5377755e-03, -1.0182157e-02, -7.9458747e-03]], dtype=float32)>,
 <tf.Tensor: shape=(64, 16, 1), dtype=float32, numpy=
 array([[[0.06258687],
         [0.06270896],
         [0.06239355],
         ...,
         [0.06330437],
         [0.06331644],
         [0.0

In [37]:
attention_result, attention_weights = attention_layer(simple_hidden, simple_output)

In [38]:
class Decoder(keras.Model):
    def __init__ (self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True)
        self.fc = keras.layers.Dense(vocab_size)
        self.attention = Attention(self.dec_units)
    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

In [39]:
decoder = Decoder(vocab_targ_size, embedding_dim, units, BATCH_SIZE)

In [40]:
decoder(tf.random.uniform((BATCH_SIZE, 1)), simple_hidden, simple_output)

(<tf.Tensor: shape=(64, 3727), dtype=float32, numpy=
 array([[-0.00563016, -0.00031051,  0.00013611, ...,  0.00173186,
          0.00554163,  0.0011226 ],
        [-0.00565664, -0.00013238, -0.0002351 , ...,  0.00107353,
          0.00512314,  0.0015196 ],
        [-0.00522375, -0.00067919,  0.00018229, ...,  0.00192473,
          0.00495865,  0.00121993],
        ...,
        [-0.00563348, -0.00080164, -0.00041911, ...,  0.00169302,
          0.00539299,  0.00211256],
        [-0.00549585, -0.00089033, -0.00032904, ...,  0.00158492,
          0.00486909,  0.00165807],
        [-0.00556515, -0.00102758, -0.00012648, ...,  0.00123833,
          0.00518886,  0.00121592]], dtype=float32)>,
 <tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[ 0.00113923, -0.00465844, -0.00455933, ...,  0.00870538,
         -0.00070354, -0.00083257],
        [ 0.00187236, -0.00442646, -0.00352619, ...,  0.00832067,
         -0.00153717, -0.00137283],
        [ 0.0014789 , -0.00540294, -0.00401729,

In [41]:
optimizer = keras.optimizers.Adam()
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [42]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [43]:
checkpoint_dir = 'chckpnts'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [44]:
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<strat>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [45]:
EPOCH = 10
for epoch in range(EPOCH):
    enc_hidden = encoder.initilize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        print('Epoch: ', epoch)
        print('Loss: ', batch_loss.numpy())
    checkpoint.save(file_prefix='test1')

Epoch:  0
Loss:  4.6504993
Epoch:  0
Loss:  4.771959
Epoch:  0
Loss:  4.725208
Epoch:  0
Loss:  4.4264317
Epoch:  0
Loss:  3.2801247
Epoch:  0
Loss:  4.251987
Epoch:  0
Loss:  3.7642245
Epoch:  0
Loss:  3.0847738
Epoch:  0
Loss:  2.8026917
Epoch:  0
Loss:  2.8952105
Epoch:  0
Loss:  3.015023
Epoch:  0
Loss:  3.016374
Epoch:  0
Loss:  2.9506352
Epoch:  0
Loss:  2.9943612
Epoch:  0
Loss:  2.9972167
Epoch:  0
Loss:  2.994089
Epoch:  0
Loss:  2.9249868
Epoch:  0
Loss:  2.7441268
Epoch:  0
Loss:  2.9279447
Epoch:  0
Loss:  2.7836058
Epoch:  0
Loss:  2.8666267
Epoch:  0
Loss:  2.7861228
Epoch:  0
Loss:  2.7839332
Epoch:  0
Loss:  2.636765
Epoch:  0
Loss:  2.748929
Epoch:  0
Loss:  2.7458498
Epoch:  0
Loss:  2.7463543
Epoch:  0
Loss:  2.6621003
Epoch:  0
Loss:  2.624762
Epoch:  0
Loss:  2.5128038
Epoch:  0
Loss:  2.4609804
Epoch:  0
Loss:  2.637305
Epoch:  0
Loss:  2.4807222
Epoch:  0
Loss:  2.4728024
Epoch:  0
Loss:  2.4210694
Epoch:  0
Loss:  2.6067138
Epoch:  0
Loss:  2.6067238
Epoch:  0
L

In [46]:
def evaluate(sentence):
    sentence = preprocess_senetence(sentence)
    inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<strat>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += target_lang_tokenizer.index_word[predicted_id] + ' '
        if target_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [47]:
checkpoint.restore(tf.train.latest_checkpoint(''))

In [48]:
evaluate('hace mucho frio aqui.')

('it s very cold . <end> ', '<strat> hace mucho frio aqui . <end>')